In [ ]:
%load_ext autoreload
%autoreload 2

Download if file is not in folder.

In [ ]:
from litedict import SQLDict

In [ ]:
TEST_1 = "key_test_1"
TEST_2 = "key_test_2"

Basic functionality

In [ ]:
d = SQLDict(":memory:")

d[TEST_1] = "asdfoobar"

assert d[TEST_1] == "asdfoobar"

del d[TEST_1]

assert d.get(TEST_1, None) is None

Glob matching

In [ ]:
d[TEST_1] = "asdfoobar"

d[TEST_2] = "foobarasd"

d["key_testx_3"] = "barasdfoo"

assert d.glob("key_test*") == ["asdfoobar", "foobarasd", "barasdfoo"]

assert d.glob("key_test_?") == ["asdfoobar", "foobarasd"]

assert d.glob("key_tes[tx]*") == ["asdfoobar", "foobarasd", "barasdfoo"]

Numbers

In [ ]:
d[TEST_1] = 1

d[TEST_2] = 2

assert d[TEST_1] + d[TEST_2] == 3

In [ ]:
with d.transaction():
    d["asd"] = "efg"
    d["foo"] = "bar"
    assert d.conn.in_transaction

In [ ]:
try:
    with d.transaction():
        d["failed"] = "no"

        assert d.conn.in_transaction

        raise Exception
except:
    # check the transaction succesfully rolled back
    assert d.get("failed") is None

In [ ]:
for k, v in d.items():
    print(k, v)

key_testx_3 "barasdfoo"
key_test_1 1
key_test_2 2
asd "efg"
foo "bar"


Test moving from/to disk/memory.

In [ ]:
import os
import pickle

fname = "test_disk.db"

d = SQLDict(
    ":memory:",
    encoder=lambda x: pickle.dumps(x).hex(),
    decoder=lambda x: pickle.loads(bytes.fromhex(x)),
)

d["foo"] = "bar"
d["asd"] = 2

d.to_disk("test_disk.db")

assert fname in os.listdir()
assert "foo" in d
assert "asd" in d

d.close()
del d


d = SQLDict(
    fname,
    encoder=lambda x: pickle.dumps(x).hex(),
    decoder=lambda x: pickle.loads(bytes.fromhex(x)),
)

assert d["foo"] == "bar"
assert d["asd"] == 2

d.to_memory()

os.unlink(fname)

assert d["foo"] == "bar"
assert d["asd"] == 2

## Benchmarks

In [ ]:
from string import ascii_lowercase, printable
from random import choice
import random


def random_string(string_length=10, fuzz=False, space=False):
    """Generate a random string of fixed length """
    letters = ascii_lowercase
    letters = letters + " " if space else letters
    if fuzz:
        letters = printable
    return "".join(choice(letters) for i in range(string_length))

In [ ]:
import gc

import pickle

import json

**Pickle**

In [ ]:
d = SQLDict(
    ":memory:",
    encoder=lambda x: pickle.dumps(x).hex(),
    decoder=lambda x: pickle.loads(bytes.fromhex(x)),
)

In [ ]:
gc.collect()

152

In [ ]:
%%timeit -n20000 -r10

d[random_string(8)] = random_string(50)

d.get(random_string(8), None)

36.8 µs ± 928 ns per loop (mean ± std. dev. of 10 runs, 20,000 loops each)


**Noop**

In [ ]:
d = SQLDict(
    ":memory:",
    encoder=lambda x: x,
    decoder=lambda x: x,
)

In [ ]:
gc.collect()

3

In [ ]:
%%timeit -n20000 -r10

d[random_string(8)] = random_string(50)

d.get(random_string(8), None)

35 µs ± 941 ns per loop (mean ± std. dev. of 10 runs, 20,000 loops each)


**JSON**

In [ ]:
d = SQLDict(
    ":memory:",
    encoder=lambda x: json.dumps(x),
    decoder=lambda x: json.loads(x),
)

In [ ]:
gc.collect()

3

In [ ]:
%%timeit -n20000 -r10

d[random_string(8)] = random_string(50)

d.get(random_string(8), None)

35.9 µs ± 707 ns per loop (mean ± std. dev. of 10 runs, 20,000 loops each)


**Pickle Python obj**

In [ ]:
d = SQLDict(
    ":memory:",
    encoder=lambda x: pickle.dumps(x).hex(),
    decoder=lambda x: pickle.loads(bytes.fromhex(x)),
)

In [ ]:
gc.collect()

3

In [ ]:
class C:
    def __init__(self, x):
        self.x = x

    def pp(self):
        return x

    def f(self):
        def _f(y):
            return y * self.x ** 2

        return _f

In [ ]:
%%timeit -n20000 -r10

d[random_string(8)] = C(random.randint(1, 200))

d.get(random_string(8), None)

20.8 µs ± 574 ns per loop (mean ± std. dev. of 10 runs, 20,000 loops each)


**Dictionary**

In [ ]:
d = {}

In [ ]:
gc.collect()

3

In [ ]:
%%timeit -n20000 -r10

d[random_string(8)] = random_string(50)

d.get(random_string(8), None)

26 µs ± 574 ns per loop (mean ± std. dev. of 10 runs, 20,000 loops each)
